In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy  as np
import matplotlib.pyplot as plt
from edesdetectrl.dataloaders.echonet import Echonet
from jax import random

# Plotting samples from Echonet
The first frame of 15 random videos.

In [ ]:
import matplotlib.pyplot as plt

videos = []
for i, data_item in enumerate(Echonet("TRAIN").get_random_generator(random.PRNGKey(42))):
    videos.append(data_item.video)
    if i == 14:
        break

fig, ax = plt.subplots(ncols=5, nrows=3, figsize=(10,6))
for i in range(5):
    for j in range(3):
        ax[j, i].imshow(videos[i+j*5][0], aspect="auto", cmap="gray")
        ax[j, i].axis('off')

fig.tight_layout()

In [ ]:
len(Echonet("TRAIN")) , len(Echonet("VAL")) , len(Echonet("TEST"))

In [ ]:
1004/sum([5891, 1009, 1004])

In [ ]:
9952-7904

# Videos with too many heartbeats

In [ ]:
from scipy.ndimage import gaussian_filter1d

echonet = Echonet(None)

bad_files = []
pp = []
for i, filename in enumerate(echonet.keys):
    try:
        f, args = echonet[filename]
        traces = echonet.volumetracings_df.loc[filename]
        ed, es = int(traces.iloc[0]["Frame"]), int(traces.iloc[-1]["Frame"])
        v = f(*args)
        video = [frame for frame in (v.video[ed:es] if ed < es else v.video[es:ed])]
        ff = video[0]
        diff = np.sum((video - ff) ** 2, axis=(1, 2))
        diff = gaussian_filter1d(diff, sigma=1)
        peaks = find_peaks(diff)
        pp.append(len(peaks[0]))
        if len(peaks[0]) >= 2:
            print(peaks, filename)
            bad_files.append(filename)
    except Exception as e:
        print(filename)
        print(e)
        break

### Let's plot some of them

In [ ]:
# Color palette
pink = "#FF00C0"
purple = "#9401FF"
dark_blue = "#4900FF"
blue = "#01B9FF"
cyan = "#00FFF9"

In [ ]:
def foo(filename):
    f, args = echonet[filename]
    traces = echonet.volumetracings_df.loc[filename]
    ed, es = int(traces.iloc[0]["Frame"]), int(traces.iloc[-1]["Frame"])
    v = f(*args)
    video = [frame for frame in (v.video[ed:es] if ed < es else v.video[es:ed])]
    ff = video[0]
    diff = np.sum((video - ff) ** 2, axis=(1, 2))
    return gaussian_filter1d(diff, sigma=1), video

import gif
@gif.frame
def frame(i, im, phase_curve):
    fig, ax = plt.subplots(ncols=2)
    ax[0].imshow(im, aspect="auto", cmap="gray")
    ax[1].plot(phase_curve)
    ax[1].axvline(i)
    fig.suptitle(i)
    fig.tight_layout()

for filename in bad_files:
    phase_curve, video = foo(filename)
    gif.save([frame(i, im, phase_curve) for (i, im) in enumerate(video)], f"bad_videos/{filename}.gif", duration=40)



In [ ]:
f"{len(bad_files)} / {len(echonet.keys)}"

In [ ]:
[k in bad_files for k in echonet.keys[:5]]

In [ ]:
bad_phase_curve, _ = foo("0X100009310A3BD7FC.avi")

fig, ax = plt.subplots(ncols=5, nrows=2, figsize=(10, 4))
for i, filename in enumerate(echonet.keys[:5]):
    ax[0, i].plot(foo(filename)[0], color=dark_blue)
    ax[1, i].plot(foo(bad_files[i+50])[0], color=dark_blue)
    ax[0, i].set_xticklabels([])
    ax[0, i].set_yticklabels([])
    ax[1, i].set_xticklabels([])
    ax[1, i].set_yticklabels([])
    ax[0, i].set_xticks([])
    ax[0, i].set_yticks([])
    ax[1, i].set_xticks([])
    ax[1, i].set_yticks([])
ax[0,0].set_ylabel("Less than two peaks", color=dark_blue)
ax[1,0].set_ylabel("Two or more peaks", color=dark_blue)
for ax in fig.axes:
    for spine in ax.spines.values():
        spine.set_edgecolor(dark_blue)
fig.tight_layout()


# FPS histograms

In [ ]:
import pandas as pd

echonet = Echonet(None)
filelist_df = pd.read_csv(echonet.filelist_csv_file)
fig = plt.figure(figsize=(5,5))
_,_,patches=plt.hist(filelist_df["FPS"], bins=40, color=blue)
plt.yscale("log")
plt.xlabel("FPS", color=dark_blue)
plt.ylabel("Number of videos (log scale)", color=dark_blue)
plt.annotate("← 50 FPS", (53,5000), color=dark_blue)
plt.tick_params(color=dark_blue, labelcolor=dark_blue)
for ax in fig.axes:
    for spine in ax.spines.values():
        spine.set_edgecolor(dark_blue)
patches[10].set_color(pink)
plt.tight_layout()
None